In [7]:
                        import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (15,30)

In [8]:
! head -10 ../data/classification_train.tsv

120GB Hard Disk Drive with 3 Years Warranty for Lenovo Essential B570 Laptop Notebook HDD Computer - Certified 3 Years Warranty from Seifelden	3950	8
"TOSHIBA SATELLITE L305-S5919 LAPTOP LCD SCREEN 15.4"" WXGA CCFL SINGLE SUBSTITUTE REPLACEMENT LCD SCREEN ONLY. NOT A LAPTOP"	35099	324
Hobby-Ace Pixhawk PX4 RGB External LED Indicator USB Module for Pixhawk Flight Controller	21822	510
Pelicans mousepad	44629	260
P4648-60029 Hewlett-Packard Tc2100 System Board	42835	68
Ectaco EI900 SD Card English - Italian	249	6
Zippered Pocket Black School Laptop Tablet Dual Straps Deluxe Backpack	4342	172
GP Mtr, 3 Ph, ODP, 3 HP, 1800 rpm, 182TC	42618	616
DIGI CM 16 PORT RJ-45 CONSOLE SERVER - Model#: 70001910	37012	248
Lenovo Ibm Thinkpad R52-1859 Laptop AC adapter, power adapter Replacement	2024	313


In [9]:
! head -10 ../data/classification_blind_set.tsv

head: cannot open '../data/classification_blind_set.tsv' for reading: No such file or directory


## Read raw data as lines

In [10]:
raw_train = pd.DataFrame([line for line in open('../data/classification_train.tsv',encoding='utf8')],columns=['line'])

In [11]:
raw_test = pd.DataFrame([line for line in open('../data/classification_blind_set_corrected.tsv',encoding='utf8')],columns=['line'])

## Extract category and brand from raw data

In [12]:
train = raw_train.line.str.extract(r'(.*)\t(\d+)\t(\d+)$',expand=True)
train.columns = ['product_title', 'brand_id', 'category_id']
train = train.dropna()
train.loc[:, ['brand_id', 'category_id']] = train.loc[:, ['brand_id', 'category_id']].astype(int)

In [13]:
test = raw_test.line.str.extract(r'(.*)\t(-?\d+)$',expand=True)
test.columns = ['product_title', 'category_id']
test = test.dropna()
test.loc[:, ['category_id']] = test.loc[:, ['category_id']].astype(int)

In [14]:
! wc -l ../data/classification_train.tsv

1000000 ../data/classification_train.tsv


In [15]:
! wc -l ../data/classification_blind_set.tsv

wc: ../data/classification_blind_set.tsv: No such file or directory


In [16]:
train.shape, test.shape

((999996, 3), (619240, 2))

# missed rows

In [17]:
1000000 - train.shape[0], 619243 -  test.shape[0]

(4, 3)

In [18]:
pd.options.display.max_colwidth = 900

First submission begin

In [19]:
category_wise_counts = train.groupby(['category_id','brand_id']).size().reset_index()
category_wise_counts.columns = ['category_id','brand_id','size']
category_wise_counts= category_wise_counts.sort_values(by=['category_id','size'],ascending=False)

In [20]:
category_wise_popular_brand = category_wise_counts.drop_duplicates(subset=['category_id'])
category_wise_popular_brand = category_wise_popular_brand.set_index('category_id')
category_wise_popular_brand.head()

,brand_id,size
category_id,,
704,25003,2024
703,5723,1
701,6526,1
700,20381,1
698,9138,2


In [21]:
category_wise_popular_brand.ix[367]

brand_id    31194
size         7042
Name: 367, dtype: int64

In [22]:
train.brand_id.value_counts().iloc[0]

41745

In [23]:
category_wise_popular_brand_submission = category_wise_popular_brand.ix[test['category_id'],'brand_id'].fillna(41745).astype(int)

In [24]:
category_wise_popular_brand_submission.shape

(619240,)

In [25]:
category_wise_popular_brand_submission.to_csv('category_wise_popular_brand_submission.csv',index=False)

End first submission

In [1]:
raw_train[~raw_train.line.str.contains(r'(.*)\t(\d+)\t(\d+)$')]

NameError: name 'raw_train' is not defined

## exploring category and brand cardinalities

In [ ]:
all_data = pd.concat([train, test])

In [ ]:
all_data.category_id.value_counts().shape

In [ ]:
vc = train.brand_id.value_counts()
vc[vc<20].shape

# Model Learning

In [ ]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from nltk import word_tokenize

In [ ]:
class Tokenizer(object):
    def __init__(self):
        self.tokenizer = word_tokenize
        self.stop_words = set(
        ['is', 'of', 'it', 'at', 'on', 'and', 'as', 'the', 'to', 'are', 'this', 'that', 'be', 'in',
          'an', 'or','any', 'all', 'am','you','we'])

    def __call__(self, text):
        text = text.lower()
        # replace special characters
        text = re.sub(r'[^a-z0-9\s/\\_\t,\-]', '', text,flags=re.IGNORECASE)
        text = re.sub(r'[/\\_\t,-]', ' ', text,flags=re.IGNORECASE)
        # replace numbers to reduce number of features
        text = re.sub(r'\b[0-9]+\b', ' __NUMBER__ ', text) 
        # replace possible product/serial numbers
        text = re.sub(r'\b\w*\d+\w*\d?\b', ' __SERIAL__ ', text)
                
        tokens = [w for w in self.tokenizer(text) if (w not in self.stop_words and len(w)>1)]
        return tokens

In [ ]:
tokenizer = Tokenizer()
tokenizer("SUN 0066152-0409N08VG5 PULLS FROM SUN FIRE V120, 36GB SCSI 80 PIN ULTRA3 10000 RPM U160 (00661520409N08VG5)")

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=Tokenizer())
%time transformed_data = vectorizer.fit_transform(train['product_title'])

In [ ]:
cv = CountVectorizer(tokenizer=Tokenizer())
%time transformed_data = cv.fit_transform(train['product_title'])

In [ ]:
len(cv.vocabulary_)

In [ ]:
wordVec = pd.Series(cv.vocabulary_)
words = wordVec.index.to_series()

In [ ]:
words = wordVec.index.to_series()
ln = words.str.len()
ln[ln>15]

In [ ]:
temp = train[train.product_title.str.contains("accessories*aradise", case=False)]
temp[temp.product_title.apply(lambda x: 'accessoriesaradise' in tokenizer(x))]